# Run Custom JavaScript Code
The following shows how to run custom JavaScript code from python and get the result. 
Since the eval runs in `NiivueModel.processCommand` in the TypeScript end, any variable accessible there is accessible through `nv.run_custom_code`.

For example, `niivue` refers to the NiiVue library and `this.nv` refers to the initialized niivue.Niivue object in the NiivueModel.

### Installation
This notebook can be run on a local server or on a virtualized environement like Google Colab.

In [ ]:
#run if using Google Colab or not building from source
try:
    # -q silence the output from pip. Remove this flag if you need 
    # to debug an installation issue.
    %pip install -q ipyniivue
    from google.colab import output
    output.enable_custom_widget_manager()
except:
    pass

In [ ]:
#other libraries used for this demo
%pip install -q nibabel
%pip install -q pytest

### Imports

In [ ]:
from pathlib import Path
import nibabel as nib
from nibabel.testing import data_path
import ipyniivue

### Create a NiiVue object

In [ ]:
nv = ipyniivue.Niivue()

### Get the colormaps
The expected output is a list of strings.

In [ ]:
colormaps = nv.run_custom_code('this.nv.colorMaps()')
colormaps

### Check the file extension

In [ ]:
is_pial = nv.run_custom_code('this.nv.isMeshExt("test.pial")')
print(is_pial)

### Volumes
No volumes have been added yet. The expected output is an empty list.

In [ ]:
volumes = nv.run_custom_code('this.nv.volumes')
print(volumes)

Now let's add a volume from nibabel's examples. We can check the volume id after adding it.

In [ ]:
example_4d_fname = Path(data_path) / 'example4d.nii.gz'

nv.add_volume(str(example_4d_fname))
volume_id = nv.run_custom_code('this.nv.volumes[0].id')
print(volume_id)

And let's also get the colormap for this volume.

In [ ]:
code = '''
var nv = this.nv;
function getColorMap() {
    var id = nv.volumes[0].id;
    return nv.colormap(id);
}
getColorMap();
'''
colormap_0 = nv.run_custom_code(code) or []
print(len(colormap_0))

Let's ask for a result with over 10 mb of data. The result is chunked in 5 mb chunks over the websocket. The cell below may take a few seconds or longer to run.

In [ ]:
volume = nv.run_custom_code('this.nv.cloneVolume(0)') or {}
print(len(volume))
print(type(volume))

Finally, let's use the `niivue` library.

In [ ]:
#the following returns None because it is undefined
nv.run_custom_code('niivue.NVImage.DT_RGBA32')

#the following returns 2304
nv.run_custom_code('''
var nv = new niivue.NVImage();
nv.DT_RGBA32;
''')